# Higgs using Dask and coffea

In [3]:
print("higgs")

higgs


In [17]:
import os 

print(os.getcwd())
os.chdir("/home/cms-jovyan/H-to-WW-NanoAOD-analysis/Text_files")
print(os.getcwd())

/home/cms-jovyan/H-to-WW-NanoAOD-analysis/notebooks
/home/cms-jovyan/H-to-WW-NanoAOD-analysis/Text_files


In [27]:
file_path = "Higgs_combined.txt"

In [28]:
import uproot
import awkward as ak
import dask
from dask import delayed
import dask.array as da
from hist import Hist
import matplotlib.pyplot as plt
import numpy as np



In [29]:
# ------------------------------
# Function to read ROOT file
# ------------------------------
def read_root_file(file_path, tree_name="Events", branches=None):
    with uproot.open(file_path) as f:
        tree = f[tree_name]
        data = tree.arrays(branches=branches, library="ak")
        # Flatten jagged arrays if needed
        return ak.flatten(data[branches[0]])


In [30]:

# ------------------------------
# Function to create Dask arrays
# ------------------------------
def root_files_to_dask_array(txt_file, tree_name="Events", branch="Electron_eta"):
    with open(txt_file, "r") as f:
        root_files = [line.strip() for line in f if line.strip()]
    
    # Create delayed tasks
    delayed_arrays = []
    for f in root_files:
        delayed_arr = delayed(read_root_file)(f, tree_name=tree_name, branches=[branch])
        # Convert to numpy within the delayed task
        delayed_np = delayed(ak.to_numpy)(delayed_arr)
        delayed_arrays.append(delayed_np)
    
    # Create dask arrays from delayed objects
    dask_arrays = [da.from_delayed(arr, shape=(np.nan,), dtype=np.float64) for arr in delayed_arrays]
    
    # Concatenate
    return da.concatenate(dask_arrays)


In [31]:

# ------------------------------
# Example usage
# ------------------------------
if __name__ == "__main__":
    txt_file = "Higgs_combined.txt"
    branch = "Electron_eta"
    
    # Load Electron_eta as a Dask array
    electron_eta_da = root_files_to_dask_array(txt_file, branch=branch)
    
    # Compute histogram using hist
    hist_obj = Hist.new.Reg(50, -5, 5, name="Electron_eta", label="Electron eta").Double()
    hist_obj.fill(electron_eta_da.compute())
    
    # Plot
    hist_obj.plot()
    plt.title("Histogram of Electron_eta from ROOT files")
    plt.show()

TypeError: HasBranches.arrays() got an unexpected keyword argument 'branches'